In [45]:
import os 
import matplotlib.pyplot as plt 
import numpy as np 
import cv2 
from IPython.display import clear_output
import pickle
from tqdm import tqdm
from tensorflow.keras.applications import resnet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import tensorflow as tf

In [2]:
# path = '../images/ImageNet100/train/'
# classes = os.listdir(path)

# labels = {}

# for c in classes : 
#     fig, axes = plt.subplots(1, 3, figsize=(10, 5))
#     for j in range(3) : 
#         img = plt.imread(path+c+'/'+os.listdir(path+c)[j])
#         img = cv2.resize(img, (150, 150))
#         axes[j].imshow(img)
#     plt.show()
#     label = input('Enter class label')
#     if label : 
#         labels[c] = label 
        
#     clear_output()

In [3]:
# with open('../images/dict.pkl', 'wb') as pickle_file:
#     pickle.dump(labels, pickle_file)

# print(f'Dictionary saved to {pickle_file_path}')

In [7]:
with open('../images/dict.pkl', 'rb') as pickle_file:
    labels = pickle.load(pickle_file)


In [68]:
# def preprocess(model, img_path, shape=(224, 224)) :
#     img = image.load_img(img_path, target_size=shape)
#     x = image.img_to_array(img)
#     x = np.expand_dims(x, axis=0)
#     x = model.preprocess_input(x)
#     return x 

from PIL import Image

def load_file(x):
    return  tf.constant(np.array(Image.open(x.numpy()).convert("L")))

def load_and_preprocess_image(image_path, label):
    img = tf.py_function(load_file, [image_path], [tf.uint8])[0]
    print(img)
    img.set_shape((224, 224))
#     img = image.load_img(image_path, target_size=(224, 224))  # Adjust target_size as needed
    img_array = image.img_to_array(img)
    img_array = resnet50.preprocess_input(img_array)  # Preprocess input based on the model architecture (e.g., ResNet50)
    return img_array, label

In [11]:
train_path = '../images/ImageNet100/train/'

train_img_paths = []
y_train = []

for c in labels : 
    L = os.listdir(train_path+c)
    for i in tqdm(range(len(L)), desc=f'Processing class {labels[c]} images'):
        img = image.load_img(train_path+c+'/'+L[i], target_size=(224, 224))
        img = image.img_to_array(img)
        # print(img.shape)
        if len(img.shape)==3 :
            train_img_paths.append(train_path+c+'/'+L[i])
            y_train.append(labels[c])

Processing class monkey images: 100%|██████████| 1300/1300 [00:04<00:00, 269.66it/s]


In [12]:
train_img_paths, y_train = np.array(train_img_paths), np.array(y_train)

In [13]:
val_path = '../images/ImageNet100/val/'

val_img_paths = []
y_val = []

for c in labels : 
    L = os.listdir(val_path+c)
    for i in tqdm(range(len(L)), desc=f'Processing class {labels[c]} images'):
        img = image.load_img(val_path+c+'/'+L[i], target_size=(224, 224))
        img = image.img_to_array(img)
        if len(img.shape)==3 :
            val_img_paths.append(val_path+c+'/'+L[i])
            y_val.append(labels[c])

Processing class monkey images: 100%|██████████| 50/50 [00:00<00:00, 325.15it/s]


In [14]:
val_img_paths, y_val = np.array(val_img_paths), np.array(y_val)

In [52]:
for i in train_dataset : 
    print(i[0].numpy())
    break

len(train_dataset), len(train_img_paths)

b'../images/ImageNet100/train/n01558993/n01558993_1004.JPEG'


(18200, 18200)

In [69]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_img_paths, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((val_img_paths, y_val))

# train_dataset = [(train_img_paths[i], y_train[i]) for i in range(len(train_img_paths))]

train_dataset = train_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.map(load_and_preprocess_image)

Tensor("EagerPyFunc:0", dtype=uint8, device=/job:localhost/replica:0/task:0)


NotImplementedError: in user code:

    File "<ipython-input-68-b3f8d3ec6da6>", line 18, in load_and_preprocess_image  *
        img_array = image.img_to_array(img)
    File "c:\RoboDK\Python37\lib\site-packages\keras\utils\image_utils.py", line 324, in img_to_array  **
        x = np.asarray(img, dtype=dtype)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (EagerPyFunc:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


In [ ]:
base_model = resnet50.ResNet50(include_top=True, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
num_classes = len(labels)

x = base_model.output
# x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32  # Adjust batch size as needed
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

# Prefetch the data to improve performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [ ]:
epochs = 6

model.fit(train_dataset, y_train, epochs=epochs, batch_size=64)

ValueError: `y` argument is not supported when using dataset as input.